<a href="https://colab.research.google.com/github/blacurly/MLOps/blob/main/DP_Model_Neptune_Athletes_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#pip install neptune

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import neptune
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

random_state=20250701

In [6]:
data = pd.read_csv("/content/drive/MyDrive/Quarter 4/MLOps/LakeFS/athletes_v2.csv")

In [8]:
import neptune

# Reconnect to your project and start a new run
run = neptune.init_run(
    project="blacurly/MLOps",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5MmQzN2ZjNy01ZGMzLTRkZjUtYWM1My04MzJmYzkwNTkyZWUifQ==",  # replace with your actual API token
    tags=["v2", "DP-model", "random-forest"],
    name="DPmodel-v2-tracking"
)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/blacurly/MLOps/e/MLOP-15


In [9]:
features = ['age','height','weight', 'deadlift', 'backsq', 'candj', 'snatch']
target = 'total_lift'

# Split
X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (24023, 7), Test shape: (6006, 7)


#### Differential Privacy Model

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from diffprivlib.models import LinearRegression

In [13]:
model = LinearRegression(epsilon=5.0)  # lower epsilon = stronger privacy, but lower accuracy
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/diffprivlib/models/linear_regression.py:278: PrivacyLeakWarning: Bounds parameters haven't been specified, so falling back to determining bounds from the data.
This will result in additional privacy leakage. To ensure differential privacy with no additional privacy loss, specify `bounds_X` and `bounds_y`.
  warnings.warn(


LinearRegression(accountant=BudgetAccountant(spent_budget=[(5.0, 0)]),
                 bounds_X=(array([18., 52.,  5.,  0.,  1.,  1.,  1.]),
                           array([  56.,   83.,  474., 1000.,  882.,  386.,  386.])),
                 bounds_y=(array([4.]), array([2030.])), epsilon=5.0)

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"DP Linear Regression Metrics:")
print(f"MAE  (Mean Absolute Error): {mae:.2f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R²   (Coefficient of Determination): {r2:.2f}")
print(f"ε (Epsilon Privacy Budget): {model.epsilon}")

DP Linear Regression Metrics:
MAE  (Mean Absolute Error): 41.88
RMSE (Root Mean Squared Error): 3287.27
R²   (Coefficient of Determination): 0.96
ε (Epsilon Privacy Budget): 5.0


In [16]:
run.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/blacurly/MLOps/e/MLOP-15/metadata
